<a href="https://colab.research.google.com/github/kingyeet9048/final_dsci_project/blob/master/Camshift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2 as cv
import numpy as np